In [6]:
# Necessary Imports
from tqdm import tqdm_notebook
import pathlib
import matplotlib.animation as animation
import imageio
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
import keras.layers as layers
from pathlib import Path
import time 
# 
def save_fig(fig_id,path, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(path, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Custom Metric to load the models without problems
import keras.backend as K
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
def kullback_leibler_divergence_test_2(y_true, y_pred):

    y_true = K.clip(y_true, K.epsilon(), 1)
    y_pred = K.clip(y_pred, K.epsilon(), 1)
    return K.sum(y_true * abs(K.log(y_true /  y_pred)), axis=-1)

custom_metr = custom_objects={
        "kullback_leibler_divergence_test_2":kullback_leibler_divergence_test_2,
        "dice_coef": dice_coef}

# def create_path_from_title(title):
#     path = title+"/"+title+".h5"
#     return path

# Loadcases chosen : LoadCase_7885,LoadCase_7818, LoadCase_7831, LoadCase_7852
def save_fig(fig_id,path, tight_layout=True, fig_extension="png", resolution=300):
    os.makedirs(path,exist_ok=True)
    path = os.path.join(path, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Define The Path of the Test Set
# Make a function to load the Inputs from the Files as np.array to predict from it
def create_instance(TEST_SET_PATH):
    instances = []
    input_path =os.path.join(TEST_SET_PATH,"Input")
    input_names = os.listdir(input_path)
    for i in input_names:
        instance= np.load(os.path.join(input_path,i),allow_pickle = True)
        instances.append(instance)
    return np.array(instances),input_names
def load_models(IDs,TRIALS_ROOT):
    loaded_models = []
    for ID in IDs:
        model_path  =os.path.join(TRIALS_ROOT,ID,str(ID+".h5"))
        loaded_model = tf.keras.models.load_model(model_path,custom_metr)
        loaded_models.append(loaded_model)
    return loaded_models

def load_model(ID, MODEL_ROOT):
    model_path  =os.path.join(MODEL_ROOT,ID,str(ID+".h5"))
    loaded_model = tf.keras.models.load_model(model_path,custom_metr)
    return loaded_model


In [2]:
# # Specify the Paths
# # The Data Root Dir, it contains possibly multiple folders with Loadcases
# PROJECT_ROOT_DIR = "F:/Thesis/inference_folder/data/"
# Loadcase_Folder_id = 'inference_test_solo'
# DATA_PATH = os.path.join(PROJECT_ROOT_DIR, Loadcase_Folder_id)
# MODIFIED_DATA_PATH = ""
# # The Path to safe the Modified Data to 
# MODEL_ROOT = "F:/Thesis/2020/TRIALS 2020/"
# MODEL_IDs = ["ID_302"]
# SAVE_ROOT = "F:/Thesis/2020/Kovertierer/ID_102/"
# os.makedirs(SAVE_ROOT,exist_ok=True)
# TEST_SET_PATH = "D:/Python Projects/Bachelor Thesis/fast_data_/test_set/191120_LoadCases_7813_7999/"

In [7]:
# Read in the files as a list of the individual lines of the files
def line_by_line(path_):
  as_list=[]
  f = open(path_, 'r')
  for line in f:      
      as_list.append(line.strip())
  return as_list


# OUT DATA
# Get the number of nodes in the Structure
def get_num_nodes(list_):
  numnodes_i= [i for i, s in enumerate(list_) if 'Total # of Grids (Structural)' in s]
  step1=list_[numnodes_i[0]].split()
  num_nodes=(int(step1[-1]))
  return num_nodes

# Get the number of elements in the structure
def get_num_elements(list_):
  numelements_i= [i for i, s in enumerate(list_) if 'Total # of Elements' in s]
  step1=list_[numelements_i[0]].split()
  num_elements=(int(step1[-1]))
  return num_elements


# Input Deck Data
# DON'T FORGET!: We start indexing at 0, so Node 132 for example is node_pos[131]

# Extracting the volume fraction
def get_vol_frac(list_):
  volfrac_i= [i for i, s in enumerate(list_) if 'DCONSTR  ' in s]
  volfrac_i= int(volfrac_i[0])
  step1=list_[volfrac_i].split()
  volfrac=(float(step1[-1]))
  return volfrac

# We get the position of nodes, in this case the element size is 2.5
def get_nodes_pos(list_,num_nodes):
# Defining the area, in which the nodes positions are saved (grid_area)
  grid_start_i= [i for i, s in enumerate(list_) if '  GRID Data' in s]
  pos= []
  grid_start_i=int(grid_start_i[0])+2
  grid_end_i = int(grid_start_i+int(num_nodes))
  grid_area= list_[grid_start_i:grid_end_i]
# Now we are accessing each line in the grid area and getting the nodes positions, 
# those have the string length of 8 characters, thats why we are accessing them through indexes
# Also there are some cases in which the position is not 0, but some very small number like:
# 3.553-15120, this is why we check also for "-" in the position
  for i,s in enumerate(grid_area):
  
    x_pos= s[24:32]
    y_pos= s[32:40]
    if "-" in x_pos:
      x_pos= 0
    else:
      x_pos= int(float(x_pos)/2.5)
    if "-" in y_pos:
      y_pos= 0
    else:
      y_pos= int(float(y_pos)/2.5)   
    pos.append([x_pos,y_pos])  
  return pos

def get_elements_pos(list_,num_elements,nodes_pos):
# Our Elements pos is equal to the node position of the lower left node
# Since we read in the Nodes mentioned in the Input deck, we have to substract 1 from
# the Node defined in the input deck, since we start indexing at 0
# Define the area containing the elements
  ele_start_i= [i for i, s in enumerate(list_) if '  CQUAD4 Elements' in s]
  pos= []
  ele_start_i=int(ele_start_i[0])+2
  ele_end_i = int(ele_start_i+int(num_elements))
  ele_area= list_[ele_start_i:ele_end_i]
  for i,s in enumerate(ele_area):
# Split the Nodes, which define the Element
    s= s.split()
# We only need the lower left node, which is the second node defined in the QUAD
# We substract -1 to get the right node element in our vector, since indexing
# This approach didnt work out, and is therefore commented out
    # node_2 = int(s[4])-1
    # node_2_pos=nodes_pos[node_2]
    # x_pos=node_2_pos[0]
    # y_pos=node_2_pos[1]
    in_1=int(s[3])-1
    in_2=int(s[4])-1
    in_3=int(s[5])-1
    in_4=int(s[6])-1
# The Approach with the second node doesnt work for some reason
# Thats why we just take the minimal entry for the lower left node
    min_pos= (min(nodes_pos[in_1],nodes_pos[in_2],nodes_pos[in_3],nodes_pos[in_4]))
    x_pos=min_pos[0]
    y_pos=min_pos[1]
    pos.append([x_pos,y_pos])
    
  return pos

# Takes the nodes positions and volfrac and makes a volfrac and node matrix
# The Volfrac matrix is just a matrix filled with the volume fraction
# The Node matrix is a matrix containing the sorted nodes positions
def get_nodes_volfrac_matrix(nodes_pos,volfrac):
# Shape doesn't care about indexing, since the 0th entry counts as well
  max_x =max(i[0] for i in nodes_pos)+1
  max_y =max(i[1] for i in nodes_pos)+1
  node_matrix= np.zeros((max_x,max_y))
  for i,s in enumerate(nodes_pos):
    i+=1
    x_=s[0]
    y_=s[1]
    node_matrix[x_,y_]=i

  volfrac_matrix=np.full((max_x-1,max_y-1),volfrac)
  return node_matrix.astype(int), volfrac_matrix.astype(float),max_x,max_y

# Same with the element matrix
def get_elements_matrix(elements_pos):
  max_x =max(i[0] for i in elements_pos)+1
  max_y =max(i[1] for i in elements_pos)+1 
  elements_matrix= np.zeros((max_x,max_y))
  for i,s in enumerate(elements_pos):  
    i+=1
    x_=s[0]
    y_=s[1]
    elements_matrix[x_,y_]=i
  return elements_matrix.astype(int),max_x,max_y

In [8]:
def get_el_matrix_from_path(folder_name,DATA_PATH):
  final=[]
  DATA_PATH = os.path.join(DATA_PATH, folder_name)
  paths=[]
  path_names= ['inputDeck.fem','inputDeck.disp','inputDeck.dens', 'inputDeck.out','inputDeck.strs','change.fem']
  for path_name in path_names:
    path_ = os.path.join(DATA_PATH, path_name)
    paths.append(path_)
  data_0 = []  
  for path_ in paths:
    data_read= line_by_line(path_)
    data_0.append(data_read)
# Out
  num_nodes = get_num_nodes(data_0[3])
  num_elements = get_num_elements(data_0[3])
# fem
  volfrac = get_vol_frac(data_0[5])
  nodes_pos = get_nodes_pos(data_0[0],num_nodes)
  elements_pos = get_elements_pos(data_0[0],num_elements,nodes_pos)
  elements_matrix, x_shape_elements,y_shape_elements = get_elements_matrix(elements_pos)
  nodes_matrix,volfrac_matrix,x_shape_nodes,y_shape_nodes = get_nodes_volfrac_matrix(nodes_pos,volfrac)
  return (elements_matrix)
    
# element_matrix = get_el_matrix_from_path("LoadCase_7818",DATA_PATH)

In [10]:
# def create_prediction_txt(loaded_models,TEST_SET_PATH,IDs,SAVE_ROOT,element_matrix):
# # Create the Test set
#     test_instances,test_names = create_instance(TEST_SET_PATH)
#     for i,s in enumerate(loaded_models):
#         for j,t in enumerate(test_instances):
#             DATA_FOLDER = os.path.join(SAVE_ROOT,test_names[j])
#             os.makedirs(DATA_FOLDER,exist_ok=True)
#             txt_title = str(str(test_names[j])+"Model"+str(IDs[i]))
#             prediction = s.predict(np.array([t]))
#             get_shape = np.shape(prediction[0][0])
#             print(get_shape)
#             resh = np.reshape(prediction,  (get_shape))
#             flat_pred = np.ndarray.flatten(resh)
#             element_matrix = np.rot90(element_matrix)
#             flat_ele = np.ndarray.flatten(element_matrix)
# #             merged_ = np.array([dict(x,y) for x,y in zip(resh,element_matrix)])
#             merged_2 = np.array(list(zip(flat_ele,flat_pred)))
            
# #             np.ndarray.flatten(merged_2)
#             path = os.path.join(DATA_FOLDER, txt_title + ".txt")
#             print(path)
#             sort_ = sorted(merged_2, key=lambda x: x[0])
#             np.savetxt(path, sort_, fmt='%5s')

In [1]:
def create_prediction_txt(model,DATA_PATH_RAW,SPLIT_DATA_PATH, model_id
                      ,SAVE_ROOT,converter_path):
    test_instances,test_names = create_instance(SPLIT_DATA_PATH)
    
    for i,s in enumerate(test_instances):
        no_npy = test_names[i][:-4]
        print(no_npy)
        txt_DATA_FOLDER = os.path.join(SAVE_ROOT,no_npy)
        os.makedirs(txt_DATA_FOLDER,exist_ok=True)
        txt_dir = txt_DATA_FOLDER
        with_p = os.path.join(txt_dir,"WithP")
        without_p =  os.path.join(txt_dir,"WithoutP")
        os.makedirs(with_p,exist_ok=True)
        os.makedirs(without_p,exist_ok=True)
        test_set_folder = os.path.join(DATA_PATH_RAW, no_npy)
        print(test_set_folder)
        input_deck = "inputDeck.fem"
        change_ = "change.fem"
        copyfile(test_set_folder+"\\"+input_deck, txt_dir + "\\" + input_deck)
        copyfile(test_set_folder+"\\"+change_, txt_dir + "\\" + change_)
        copyfile(converter_path+"\\"+"Avdalim_OptiStruct.exe", txt_dir + "\\" + "Avdalim_OptiStruct.exe")
        element_matrix = get_el_matrix_from_path(no_npy,DATA_PATH_RAW)
        txt_title = str(str(no_npy)+"Model"+str(model_id) + ".txt")
        t = time.time()
        prediction = model.predict(np.array([s]))
        elapsed = time.time() - t
        print("Elapsed prediction time: ", elapsed)
        get_shape = np.shape(prediction[0][0])
        resh = np.reshape(prediction,  (get_shape))
        flat_pred = np.ndarray.flatten(resh)
        element_matrix = np.rot90(element_matrix)
        flat_ele = np.ndarray.flatten(element_matrix)
        merged = np.array(list(zip(flat_ele,flat_pred)))
            
#             np.ndarray.flatten(merged_2)
        txt_path = os.path.join(txt_dir, txt_title)
        sort_ = sorted(merged, key=lambda x: x[0])
        np.savetxt(txt_path, sort_, fmt='%5s')
        as_array = np.array([[txt_title],[input_deck]])
        config_path = os.path.join(txt_dir, "config.txt")
        np.savetxt(config_path, as_array, fmt='%5s')    
        os.chdir(txt_dir)
        p = subprocess.Popen("Avdalim_OptiStruct.exe")

In [23]:

# import pathlib
# import matplotlib.pyplot as plt
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# import keras.layers as layers
# from pathlib import Path
# import matplotlib.pyplot as plt
# import os
# from tqdm import tqdm_notebook
# import nbimporter
# import data_modification_inf as Modify
# import split_data_inf as Split
# import create_output_txt as Outtxt
# import subprocess
# from shutil import copyfile


# DATA_PATH = "F:/Thesis/inference_folder/data/"
# DATA_PATH_RAW = os.path.join(DATA_PATH,"raw")
# TEMP_PATH = "F:/Thesis/inference_folder/temp/"
# os.makedirs(TEMP_PATH,exist_ok=True)
# MODEL_PATH = "F:/Thesis/inference_folder/utils/"
# model_ID = "ID_302"
# SAVE_ROOT = "F:/Thesis/inference_folder/data/out/"
# os.makedirs(TEMP_PATH,exist_ok=True)
# MODIFIED_DATA_PATH = os.path.join(TEMP_PATH,"modified")
# os.makedirs(MODIFIED_DATA_PATH,exist_ok=True)
# SPLIT_DATA_PATH = os.path.join(TEMP_PATH,"split")
# os.makedirs(SPLIT_DATA_PATH,exist_ok=True)
# CONVERTER_PATH = "F:/Thesis/inference_folder/utils/"

# import keras.backend as K
# def dice_coef(y_true, y_pred, smooth=1):
#     y_true_f = K.flatten(y_true)
#     y_pred_f = K.flatten(y_pred)
#     intersection = K.sum(y_true_f * y_pred_f)
#     return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
# def kullback_leibler_divergence_test_2(y_true, y_pred):

#     y_true = K.clip(y_true, K.epsilon(), 1)
#     y_pred = K.clip(y_pred, K.epsilon(), 1)
#     return K.sum(y_true * abs(K.log(y_true /  y_pred)), axis=-1)

# custom_metr = custom_objects={
#         "kullback_leibler_divergence_test_2":kullback_leibler_divergence_test_2,
#         "dice_coef": dice_coef}
# def load_model(ID, MODEL_ROOT):
#     model_path  =os.path.join(MODEL_ROOT,str(ID+".h5"))
#     loaded_model = tf.keras.models.load_model(model_path,custom_metr)
#     return loaded_model
# tf.keras.backend.clear_session()
# model = load_model(model_ID,MODEL_PATH)

In [24]:


# SPLIT_DATA_PATH_temp = "F:/Thesis/inference_folder/temp/split/raw/"

# DATA_PATH = "F:/Thesis/inference_folder/data/"
# DATA_PATH_RAW = os.path.join(DATA_PATH,"raw")
# print(DATA_PATH_RAW)
# create_prediction_txt(model,DATA_PATH_RAW,SPLIT_DATA_PATH_temp, model_ID
#                       ,SAVE_ROOT,CONVERTER_PATH)

F:/Thesis/inference_folder/data/raw
LoadCase_7813
F:/Thesis/inference_folder/data/raw\LoadCase_7813


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model_1/conv2d_15/Relu (defined at C:\Users\Avdalim\Anaconda3\envs\Thesis\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_distributed_function_38679]

Function call stack:
distributed_function


In [15]:
import subprocess
from shutil import copyfile
def run_converter(TEST_COMP_SET_PATH,loadcase_names,txt_folder_path,loaded_models, IDs,converter_path):
    for i,s in enumerate(loaded_models):
        for j,t in enumerate(loadcase_names):
            no_npy = t[:-4]
            txt_dir = os.path.join(txt_folder_path,t)
            with_p = os.path.join(txt_dir,"WithP")
            without_p =  os.path.join(txt_dir,"WithoutP")
            os.makedirs(with_p,exist_ok=True)
            os.makedirs(without_p,exist_ok=True)
            test_set_folder = os.path.join(TEST_COMP_SET_PATH, no_npy)
            input_deck = "inputDeck.fem"
            change_ = "change.fem"
            copyfile(test_set_folder+"\\"+input_deck, txt_dir + "\\" + input_deck)
            copyfile(test_set_folder+"\\"+change_, txt_dir + "\\" + change_)
            copyfile(converter_path+"\\"+"Avdalim_OptiStruct.exe", txt_dir + "\\" + "Avdalim_OptiStruct.exe")
            converter = str(os.path.join(txt_dir,"Avdalim_OptiStruct.exe"))
#             print(converter)
#             subprocess.call([converter])
            os.chdir(txt_dir)
            p = subprocess.Popen("Avdalim_OptiStruct.exe")
            
            txt_title = str(str(test_names[j])+"Model"+str(IDs[i])+".txt")
            b = "inputDeck.fem"
            as_array = np.array([[txt_title],[b]])
            path = os.path.join(txt_dir, "config.txt")
            np.savetxt(path, as_array, fmt='%5s')
            
            
            
            